
# Mushroom Manifest Builder

This notebook scans a folder (e.g., `TexturePack/mushroom_pack/`) for your generated PNGs
(named **`color-stem-cap-value.png`**, like `black-03-0.201-+0.png`), then writes a
`manifest.json` into that folder. Your web app will fetch this manifest at runtime.

## What it does
- Recursively scans for files ending in `.png`.
- Validates filenames against the pattern: `([a-z]+)-(\d+)-(\d+\.\d+)-([+-]?\d+)\.png`.
- Builds an array of file paths (either basenames or paths relative to the base folder).
- Saves `manifest.json` in the base folder.
- Prints helpful stats (per-color counts, a few samples).

## How to use
1. **Edit** the `BASE_DIR` below to the absolute path of your folder on *your* machine,
   e.g. `r"/Users/you/Project/TexturePack/mushroom_pack"`.
2. Run each cell top-to-bottom.
3. Confirm the printed stats look right.
4. Check that `manifest.json` exists under `BASE_DIR`.
5. In your webpage, it will be at: `TexturePack/mushroom_pack/manifest.json` (or the
   path you serve).

> Tip: If your JS expects basenames only, set `WRITE_BASENAMES = True`. If you prefer
> manifest entries to include the folder path, set it to `False` to write paths **relative**
> to `BASE_DIR`.


In [1]:
# === Use your exact local folder for scanning (only used to FIND files) ===
BASE_DIR = r"/Users/jianleguo/Documents/GitHub/vmps_mario/TexturePack/mushroom_pack"

# Write only filenames like "black-03-0.201-+0.png" (no paths in manifest)
WRITE_BASENAMES = True

# Name of the output manifest placed INSIDE BASE_DIR
MANIFEST_NAME = "manifest.json"

import os
assert os.path.isdir(BASE_DIR), f"BASE_DIR does not exist: {BASE_DIR}"
print("✓ Using BASE_DIR:", BASE_DIR)
print("Manifest will contain basenames only:", WRITE_BASENAMES)


✓ Using BASE_DIR: /Users/jianleguo/Documents/GitHub/vmps_mario/TexturePack/mushroom_pack
Manifest will contain basenames only: True


In [2]:

import os, re, json
from collections import Counter

pattern = re.compile(r"^([a-z]+)-(\d+)-(\d+\.\d+)-([+-]?\d+)\.png$", re.I)

def basename(p: str) -> str:
    return os.path.basename(p)

def is_valid_name(name: str) -> bool:
    return pattern.match(name) is not None

def scan_pngs(base_dir: str, write_basenames: bool = True):
    found = []
    invalid = []
    for root, _, files in os.walk(base_dir):
        for f in files:
            if not f.lower().endswith(".png"):
                continue
            if is_valid_name(f):
                if write_basenames:
                    found.append(f)
                else:
                    # path relative to base_dir
                    rel = os.path.relpath(os.path.join(root, f), base_dir)
                    found.append(rel.replace("\\", "/"))
            else:
                invalid.append(f)
    return found, invalid

def color_from_name(name: str) -> str:
    m = pattern.match(name)
    return m.group(1).lower() if m else "invalid"


In [3]:

# --- Scan ---
assert os.path.isdir(BASE_DIR), f"BASE_DIR does not exist: {BASE_DIR}"
found, invalid = scan_pngs(BASE_DIR, write_basenames=WRITE_BASENAMES)

print(f"Scanned folder: {BASE_DIR}")
print(f"Valid PNGs found: {len(found)}")
print(f"Invalid (skipped): {len(invalid)}")
if invalid:
    print("Some invalid samples:", invalid[:10])

# --- Stats ---
by_color = Counter(color_from_name(os.path.basename(x)) for x in found)
print("\nCounts by color:", dict(by_color))

print("\nSample entries:", found[:10])

# --- Save manifest ---
manifest_path = os.path.join(BASE_DIR, MANIFEST_NAME)
with open(manifest_path, "w", encoding="utf-8") as f:
    json.dump(found, f, ensure_ascii=False, indent=2)

print(f'\nWrote manifest with {len(found)} entries -> {manifest_path}')


Scanned folder: /Users/jianleguo/Documents/GitHub/vmps_mario/TexturePack/mushroom_pack
Valid PNGs found: 3200
Invalid (skipped): 0

Counts by color: {'green': 400, 'black': 400, 'white': 400, 'magenta': 400, 'yellow': 400, 'cyan': 400, 'red': 400, 'blue': 400}

Sample entries: ['green-14-0.912-+3.png', 'black-10-0.234-+3.png', 'white-08-1.365--1.png', 'magenta-04-2.000--6.png', 'yellow-03-0.203-+2.png', 'green-13-1.112-+3.png', 'green-03-1.290--1.png', 'cyan-07-1.123-+5.png', 'red-06-1.145-+5.png', 'cyan-14-0.207-+1.png']

Wrote manifest with 3200 entries -> /Users/jianleguo/Documents/GitHub/vmps_mario/TexturePack/mushroom_pack/manifest.json


In [4]:

# Re-open and validate the just-written manifest
manifest_path = os.path.join(BASE_DIR, MANIFEST_NAME)
with open(manifest_path, "r", encoding="utf-8") as f:
    data = json.load(f)

assert isinstance(data, list), "Manifest JSON must be an array."
bad = [x for x in data if not is_valid_name(os.path.basename(x))]
print(f"Re-loaded {len(data)} entries from manifest.")
print("Invalid names in manifest (should be 0):", len(bad))
if bad:
    print(bad[:10])


Re-loaded 3200 entries from manifest.
Invalid names in manifest (should be 0): 0
